# Implementing Rule-based Model

In [1]:
import numpy as np
import pandas as pd
import textacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from spacy import displacy

- universal dependencies: https://universaldependencies.org/en/dep/index.html
- aspect-based opinion mining: https://devopedia.org/aspect-based-opinion-mining
- Apect extration GT: https://achyutjoshi.github.io/aspect_extraction/aspectextraction
    - https://github.com/ishikaarora/Aspect-Sentiment-Analysis-on-Amazon-Reviews/blob/master/src/models/aspect_extraction.py
- https://www.researchgate.net/publication/327707096_Aspect_Extraction_Performance_with_POS_Tag_Pattern_of_Dependency_Relation_in_Aspect-based_Sentiment_Analysis


In [2]:
lang_module = 'en_core_web_sm'
corpus = textacy.Corpus.load(lang=lang_module, filepath='./data/bank_reviews.bin.gz')

Notes:
- This approach may not be context aware. When two different products are beind discussed the Aspect/Modifier may not point to the correct product.
- Implicit vs. Explicit, our rule-based approach can only deal with explicit. Implicit aspect-opinion is missed
- Attention on aspects can be missed if not directly connected by dependency
- Punctuation changes meanings of A/M -> line 14 "Good bank, great service" vs. without comma. Co-operation split into two "co" and "operation"
- Very restrictive, can not keep up with the many ways people can write text

-> biggest concern how to evaluate un-labeled data

# Writting dependency rules to extract Aspects & Modifiers

In [3]:
# Start analyzer
analyzer = SentimentIntensityAnalyzer()

In [109]:
ITEM = 30
doc = corpus[ITEM]

product_references = ['it','this','they','these']


def add_compound(token, aspect):
    for child in token.children:
        # if child is compound then add as prefix
        if child.dep_ == 'compound' and aspect != '99999':
            aspect = child.norm_ + " " + aspect
        # if child clause modifier add to Aspect
        if child.dep_ == 'relcl':
            aspect = child.norm_ + " " + aspect
    return aspect

def add_negative(token, modifier):
    if(token.dep_ == "neg" and modifier != "99999"):
        neg_prefix = token.norm_ if token.norm_ != "n't" else "not"
        modifier = neg_prefix + " " + modifier
    if(token.dep_ == "det" and token.norm_ == 'no' and modifier != "99999"):
        neg_prefix = 'no'
        modifier = neg_prefix + " " + modifier
    # example - 'this could have been better' -> (this, not better)
    # if(token.dep_ == "aux" and token.tag_ == "MD" and modifier != "99999"):
    #     neg_prefix = "not"
    #     modifier = neg_prefix + " " + modifier
    return modifier

# for i, doc in enumerate(corpus[:40]):

def get_aspect_level_sentiments(doc:object):
    # print(doc._.meta['product'])
    # print(i,doc)

    ## FIRST RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    ## RULE = M is child of A with a relationship of amod
    rule_pairs = []

    for token in doc:
        A = "99999"               # aspect
        M = "99999"               # modifier
        if token.dep_ == "amod" and not token.is_stop:
            # if token.pos_ in ()
            M = token.norm_
            A = token.head.norm_
            if token.head.dep_ == 'dobj':
                # print(token.head.head)
                if token.head.head.dep_ == 'advcl':
                    # print(token.head.head.head)
                    M = token.head.head.head.norm_ + " " + token.head.head.norm_ + " " + M
                else:
                    M = token.head.head.norm_ + " " + M
            for child_m in token.head.head.children:
                M = add_negative(child_m, M)
                for child_m2 in child_m.children:
                    M = add_negative(child_m2, M)

            # advervial modifiers (most refreshing lotion)
            M_children = token.children
            for child_m in M_children:
                if (child_m.dep_ == "advmod"):
                    M_hash = child_m.norm_
                    M = M_hash + " " + M
                    # break
                M = add_negative(child_m, M)

            # negation in adjective, the "no" keyword is a 'det' of the noun (e.g. no interesting characters)
            A = add_compound(token.head, A)
            # A_children = token.head.children
            # for child_a in A_children:
            #     # if child is compound then add as prefix or suffix(try to determine rule for this)
            #     if child_a.dep_ == 'compound':
            #         A = child_a.norm_ + " " + A
                
            #     # if child clause modifier add to Aspect
            #     if child_a.dep_ == 'relcl':
            #         A = child_a.norm_ + " " + A

                # # add negative
                # if(child_a.dep_ == "det" and child_a.norm_ == 'no'):
                #     neg_prefix = 'not'
                #     M = neg_prefix + " " + M
                #     break

        if(A != "99999" and M != "99999"):
            rule_pairs.append((A, M, analyzer.polarity_scores(M + " " + A)['compound'],1))



    ## TWO RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    ## Adjectival Complement - A is a child of something with relationship of nsubj, while
    ## M is a child of the same something with relationship of acomp
    ## Assumption - A verb will have only one NSUBJ and DOBJ
    ## "The sound of the speakers would be better. The sound of the speakers could be better" - handled using AUX dependency
    for token in doc:

        children = token.children
        A = "99999"
        M = "99999"
        add_neg_pfx = False
        for child in children :
            if(child.dep_ == "nsubj" and not child.is_stop):
                A = child.norm_
                # check_spelling(child.norm_)
                for child_two in child.children:
                    if child_two.dep_ == "compound":
                        A = child_two.norm_ + " " + A

            if(child.dep_ == "acomp" and not child.is_stop):
                M = child.norm_
                children_two = child.children
                for child_two in children_two:
                    if child_two.dep_ == 'advmod':
                        M = child_two.norm_ + " " + child.norm_
                    else:
                        M = child.norm_
            M = add_negative(child, M)
        #     # example - 'this could have been better' -> (this, not better)
        #     if(child.dep_ == "aux" and child.tag_ == "MD"):
        #         neg_prefix = "not"
        #         add_neg_pfx = True

        #     if(child.dep_ == "neg"):
        #         neg_prefix = child.norm_
        #         add_neg_pfx = True

        # if (add_neg_pfx and M != "99999"):
        #     M = neg_prefix + " " + M
        #         #check_spelling(child.norm_)

        if(A != "99999" and M != "99999"):
            rule_pairs.append((A, M, analyzer.polarity_scores(M + " " + A)['compound'],2))


    ## THIRD RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect
    ## ATTR - link between a verb like 'be/seem/appear' and its complement
    ## Example: 'this is garbage' -> (this, garbage)
    for token in doc:
        children = token.children
        A = "99999"
        M = "99999"
        add_neg_pfx = False
        for child in children :
            if(child.dep_ == "nsubj" and not child.is_stop):
                A = child.norm_
                # check_spelling(child.norm_)

            if((child.dep_ == "attr") and not child.is_stop):
                M = child.norm_
                #check_spelling(child.norm_)

            M = add_negative(child, M)
        #     if(child.dep_ == "neg"):
        #         neg_prefix = child.norm_
        #         add_neg_pfx = True

        # if (add_neg_pfx and M != "99999"):
        #     M = neg_prefix + " " + M

        if(A != "99999" and M != "99999"):
            rule_pairs.append((A, M,analyzer.polarity_scores(M + " " + A)['compound'],3))        


    ## FOURTH RULE OF DEPENDANCY PARSE -
    ## M - Sentiment modifier || A - Aspect

    #Adverbial modifier to a passive verb - A is a child of something with relationship of nsubjpass, while
    # M is a child of the same something with relationship of advmod

    #Assumption - A verb will have only one NSUBJ and DOBJ
    for token in doc:


        children = token.children
        A = "99999"
        M = "99999"
        add_neg_pfx = False
        for child in children :
            if((child.dep_ == "nsubjpass" or child.dep_ == "nsubj") and not child.is_stop):
                A = child.norm_
                # check_spelling(child.norm_)

            if(child.dep_ == "advmod" and not child.is_stop):
                M = child.norm_
                M_children = child.children
                for child_m in M_children:
                    if(child_m.dep_ == "advmod"):
                        M_hash = child_m.norm_
                        M = M_hash + " " + child.norm_
                        # break
                #check_spelling(child.norm_)
            M = add_negative(child, M)
        #     if(child.dep_ == "neg"):
        #         neg_prefix = child.norm_
        #         add_neg_pfx = True

        # if (add_neg_pfx and M != "99999"):
        #     M = neg_prefix + " " + M

        if(A != "99999" and M != "99999"):
            rule_pairs.append((A, M,analyzer.polarity_scores(M + " " + A)['compound'],4)) # )

    # FIFTH RULE 
    # A: Direct Object with NOUN type
    # M: Adj or Modifier of some sort
    # # Assumes that all direct objects with adjectives pointing may compose a A-M relationship
    for token in doc:
        A = "99999"               # aspect
        M = "99999"               # modifier
        # Adding aspect
        if token.dep_ == 'dobj' and token.pos_ == 'NOUN':
            A = token.norm_
        # Adding compound to aspect 
        A = add_compound(token=token, aspect=A)
        # Get modifier if matching dep and pos
        for child in token.children:
            if child.dep_ in ('advmod') and child.pos_ in ('ADV'):
                M = child.norm_
            M = add_negative(child, M)
            
        if(A != "99999" and M != "99999"):
            rule_pairs.append((A, M, analyzer.polarity_scores(M + " " + A)['compound'],5))
    
    # Removing pairs that do not have sentiment
    rule_pairs = [(A,M,P,r) for (A,M,P,r) in rule_pairs if P != 0]
    # print(f'\t {rule_pairs}\n')
    return rule_pairs

# Example Row 0
# DONE -> Try adding case for CC "helped" (mod) build-credit (aspect) 

# Example Row 2
# DONE -> Neither should be captured [('credit card', 'new', 0.0, 1), ('future', 'near', 0.0, 1)] -> dismiss if 0 sentiment?

# Row 9
# DONE -> Misses cash back is a bonus 


# TODO:
# 1. IMPROVE ALGO
# DONE Capture negatives correctly - line 30
# Row 10
# M:help, A:credit back-up
# Example Row 3
# Missed 0 percent interest, good for balance -> A: select obj type (add compound if present) M: head -> get adj 

results = list()
for i, doc in enumerate(corpus):
    aspects = get_aspect_level_sentiments(doc)
    results.append(aspects)
len(results)

23287

In [107]:
choice = 3226
print(choice, corpus[choice])
print(get_aspect_level_sentiments(corpus[choice]), '\n')

3226 This was the first credit card that I have ever received. It was a pleasant surprise to actually be approved for the card in the first place seeing that I was a new college student with no prior credit, and to make matters worse it was shortly after the economic collapse and no one was really giving out credit.After receiving the card it was everything that I could have asked for. I was started on a basic 500$ limit, which was all that I was really looking for. I also did have a points based reward program which turned out to be very beneficial. Every experience with Bank of America has been positive, and there is really nothing negative that I could say.
[('approved surprise', 'pleasant', 0.802, 1), ('credit', 'no prior', 0.1027, 1), ('collapse', 'economic', -0.4939, 1), ('turned reward program', 'have based', 0.5719, 1), ('say nothing', 'negative', -0.5719, 1), ('experience', 'positive', 0.5574, 2)] 



In [99]:
# Visualize an example of dependency
displacy.render(corpus[choice], style='dep', jupyter=True, options={'distance':120})

In [58]:
analyzer.polarity_scores('annual fee')['compound']

0.0

In [7]:
# doc = corpus[ITEM]
# # for doc in corpus[:1]:
# #     print(doc)
# for token in doc:
#     # if token.dep_ == 'ccomp':
#     print(token.text, token.pos_, token.norm_, token.tag_ ,token.dep_, [child.text for child in token.children], [parent.text for parent in token.ancestors])

## Sample for Evaluation

In [27]:
print(corpus[1], corpus[1]._.meta)

Stay as far away from this lender as possible. My mortgage got sold to pnc last month and it has been nothing but a nightmare trying to get my payment through. I paid my $1000 on the Friday of last month. The following Monday it was taken out of my checking. The next day another $1000 was taken out. I had to transfer money so my account wouldn’t overdraft. After days of trying to talk to customer service and getting the payment stopped I called today to have Brandy the customer service rep laugh and say ‘you’ve never had a payment for something get taken out twice?’ This place is a nightmare. {'date': '2021-09-07', 'year': 2021, 'bank': 'PNC', 'product': 'Mortgages', 'stars': 1}


In [123]:
np.random.seed(42)
# choices = np.random.choice(range(len(corpus)),size=30)
# choices
choices = [1603, 3226, 13381, 3189, 17654, 19797, 19639, 6077, 5168, 7417, 22421, 21576, 428, 501, 1201, 14108, 8679, 22716, 9876, 367, 20665, 16677, 15017, 167, 5339, 22228, 16510, 2789, 7403, 12701]

for choice in choices:
    print(choice, corpus[choice])
    print(get_aspect_level_sentiments(corpus[choice]), '\n')

1603 They try to charge me $5 to replace my debit card that’s not working and my card keeps getting declined everywhere when I have the funds available in my account.
[] 

3226 This was the first credit card that I have ever received. It was a pleasant surprise to actually be approved for the card in the first place seeing that I was a new college student with no prior credit, and to make matters worse it was shortly after the economic collapse and no one was really giving out credit.After receiving the card it was everything that I could have asked for. I was started on a basic 500$ limit, which was all that I was really looking for. I also did have a points based reward program which turned out to be very beneficial. Every experience with Bank of America has been positive, and there is really nothing negative that I could say.
[('approved surprise', 'pleasant', 0.802, 1), ('credit', 'no prior', 0.1027, 1), ('collapse', 'economic', -0.4939, 1), ('turned reward program', 'have based', 

# MAP Aspect Based Sentiment to DICTIONARY 
mapping common words captured by algo to match main aspects: Customer service, Interest Rate etc. #.

1. The aspects dictionary should contain all the hardcoded words we should identify for the main categories given.

2. Func aggregates sentiment score per record and puts them into a dictionary

In [111]:
aspects = {
    'Credit Starter':['repair', 'damaged','building'],
    'Customer Service':['human', 'computerized', 'banker', 'refund', 'experience', 'they','live person','talk','speaking','representative','manager', 'teller', 'banker', 'lady', 'customer', 'service', 'management', 'english', 'operation', 'agent','staff'],
    'Interest Rates': ['rate', 'interest', 'accrued'],
    'Online Banking Services': ['transactions', 'feature','website','bills', 'tool', 'monitor', 'app', 'platform', 'online', 'menu','automatic','payment'],
    'Rewards': ['bonus', 'offer', 'categorie','category', 'reward', 'point', 'mile', 'cash', 'back'],
    'Fees': ['fee', 'charge', 'charges','free', 'rebate', 'deal','discount'],
    'Security': ['fraud', 'dispute', 'security', 'seller'],
    'Retail Branch': ['branch', 'location', 'store', 'distance', 'branches', 'atm']
}

In [112]:
## Create function that will create main category scores
# if aspect falls into selected categories -extract score into records dictionary
sample = results[:500]


def aggregate_sentiment_scores(results, aspects) -> list[dict]:
    all_scores = list()
    # TODO: what if multiple mentions same aspect?

    for result in results:
        scores = {
            'Credit Starter':np.nan,
            'Customer Service': np.nan,
            'Interest Rates': np.nan,
            'Online Banking Services': np.nan,
            'Rewards':np.nan,
            'Fees':np.nan,
            'Security':np.nan,
            'Retail Branch': np.nan
        }
        # get result pair
        for pair in result:
            item = pair[0]
            # for aspect word in item
            for word in item.split():
                # for overall aspect categories
                for cat, words in aspects.items():
                    # check if word in result pair is in aspect words
                    if word.rstrip('s') in words:
                        # add score if none existent
                        if scores[cat] is np.nan:
                            scores[cat] = 0
                        scores[cat] = scores.get(cat, 0) + pair[2]
        all_scores.append(scores)
    return all_scores

    
agg_scores = aggregate_sentiment_scores(results, aspects)

In [113]:
len(agg_scores)

23287

In [114]:
results[860]

[('is establishment', 'honest', 0.5106, 1),
 ('charges', 'charged monthly', -0.4404, 1),
 ('complaint', 'not biggest', 0.2235, 1),
 ('staff', 'kind', 0.5267, 2)]

In [115]:
agg_scores[860]

{'Credit Starter': nan,
 'Customer Service': 0.5267,
 'Interest Rates': nan,
 'Online Banking Services': nan,
 'Rewards': nan,
 'Fees': -0.4404,
 'Security': nan,
 'Retail Branch': nan}

# CREATE MASTER DATAFRAME
We create a master dataframe of records and add their corresponding selected scores


| bank | product | date | text | selected aspect scores * |
-

In [116]:
def create_df_from_spacy(corpus):
    records = list()
    for doc in corpus:
        base = doc._.meta
        base['text'] = doc.text
        records.append(base)
    return records

df = create_df_from_spacy(corpus)

In [117]:
df = pd.DataFrame(df)
agg_scores = pd.DataFrame(agg_scores)

df = pd.concat([df, agg_scores], axis=1)

Our scoring function is really sparse out of 23,000 records only 6-2.6K values are filled

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23287 entries, 0 to 23286
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   date                     23287 non-null  object 
 1   year                     23287 non-null  int64  
 2   bank                     23287 non-null  object 
 3   product                  23287 non-null  object 
 4   stars                    23287 non-null  int64  
 5   text                     23287 non-null  object 
 6   Credit Starter           33 non-null     float64
 7   Customer Service         5103 non-null   float64
 8   Interest Rates           3051 non-null   float64
 9   Online Banking Services  1560 non-null   float64
 10  Rewards                  2559 non-null   float64
 11  Fees                     3268 non-null   float64
 12  Security                 289 non-null    float64
 13  Retail Branch            376 non-null    float64
dtypes: float64(8), int64(2

In [119]:
df

,date,year,bank,product,stars,text,Credit Starter,Customer Service,Interest Rates,Online Banking Services,Rewards,Fees,Security,Retail Branch
0,2021-09-08,2021,PNC,Credit Cards,5,The PNC cash rewards card is a great first cre...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-07,2021,PNC,Mortgages,1,Stay as far away from this lender as possible....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-06,2021,PNC,Credit Cards,5,I like everything about it don't want to chang...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-02,2021,PNC,Credit Cards,5,Good card....0 percent interest for first year...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-02,2021,PNC,Business Services,2,Local branch very Hard to reach by phone. Gene...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23282,2012-06-04,2012,TD Bank,Checking,5,TD Bank is the absolute best bank that I have ...,NaN,1.0691,NaN,NaN,NaN,NaN,NaN,NaN
23283,2012-06-04,2012,TD Bank,Checking,5,I have a personal checking account at TD and h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23284,2012-05-30,2012,TD Bank,Checking,5,I am a customer of TD Bank and I think they ar...,NaN,0.4215,NaN,NaN,NaN,NaN,NaN,NaN
23285,2012-05-30,2012,TD Bank,Checking,5,I have been using TD Bank for two years now. I...,NaN,NaN,0.0516,0.2716,NaN,-0.2732,NaN,NaN


In [120]:
df.iloc[23282].text

"TD Bank is the absolute best bank that I have ever done business with. The fact that it is the only major bank that is open until nine o clock and open seven days a week should tell you how much they care about their customers. It took me less than fifteen minutes to walk into a branch and start up my first account with them. The customer service is amazing and it is always a pleasure to talk to them. I'm a huge fan of the coin machine that is located in every branch. TD Bank knows how hard its customers work for their money and they make sure that every single red cent is held with the highest regard."

In [121]:
results[23282]

[('done bank', 'best', 0.6369, 1),
 ('fan', 'huge', 0.5574, 1),
 ('customer service', 'amazing', 0.5859, 2),
 ('customers', 'how hard', -0.1027, 2)]

1. It appears that the algorithm needs to be improved to accurately represent the sentiment regarding each given aspect

2. words mapping needs to be improved to capture more sentiment scores, so that we can have better representation of scores

In [122]:
df.to_csv('example_output.csv')